In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

tr = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
te = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
re = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train = tr.fillna(0)
test = te.fillna(0)

feature_names = ['GarageArea', 'GarageCars', 'TotRmsAbvGrd', 'BedroomAbvGr', 'GrLivArea',
                 '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF', 'OverallQual', 'OverallCond',
                 'MSSubClass']
X = train[feature_names]
x = test[feature_names]
Y = train['SalePrice']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=0,
)


In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label = Y)
dtest = xgb.DMatrix(x)


In [ ]:

# モデルのインスタンス作成
#model = xgb.XGBRegressor()
re = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
watchlist = [(dtrain, 'train')]
num_round = 50
import optuna
from sklearn.metrics import mean_squared_error

def objective(trial):
    params = {'objective':'reg:squarederror', 
              #'booster':'dart',
              'eta':0.1,
              #ここからパラメータチューニング
              'gamma':trial.suggest_loguniform('gamma', 1e-8, 1.0),
              'min_child_weight':trial.suggest_loguniform('min_child_weight',0.1, 10), 
              'max_depth':trial.suggest_int('max_depth',3,9), 
              'subsumple':trial.suggest_discrete_uniform('subsumple',0.6,0.95,0.05),
              'colsumple_bytree':trial.suggest_discrete_uniform('colsumple_bytree',0.6,0.95,0.05)
             }
    
    model = xgb.train(params, dtrain, num_round)
    y_test_pred = model.predict(dtest)
    y_train_pred = model.predict(dtrain)
    rmse = np.sqrt(mean_squared_error(Y,y_train_pred))
    return rmse




In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials = 30)

In [ ]:
print(study.best_params)
print(study.best_value)
best_params = study.best_params

In [ ]:
best_params.setdefault('objective','reg:squarederror')
best_params.setdefault('eta',0.1)
best_params
best_model = xgb.train(best_params, dtrain, num_round)
y_test_pred = best_model.predict(dtest)
y_train_pred = best_model.predict(dtrain)
rmse = np.sqrt(mean_squared_error(Y,y_train_pred))
print(rmse)

In [ ]:
plt.plot(re['SalePrice'], y_test_pred, 'o', markersize = 1)
plt.xlabel("true")
x = [1, 300000]
plt.plot(x, x)
plt.show()
x = [1, 800000]
plt.plot(x, x)
plt.plot(Y, y_train_pred, 'o', markersize = 1)



In [ ]:


submission = pd.DataFrame()
submission['Id'] = te['Id']
submission['SalePrice'] = y_test_pred
submission.set_index("Id",inplace=True)

print(submission)
submission.to_csv("HousePrice_XGB_3.csv", index_label = ["Id"])


y_train_pred = model.predict(dtrain)

In [ ]:
plt.plot(re['SalePrice'], y_test_pred, 'o', markersize = 1)
plt.xlabel("true")
x = [1, 300000]
plt.plot(x, x)
plt.show()
x = [1, 800000]
plt.plot(x, x)
plt.plot(Y, y_train_pred, 'o', markersize = 1)
